# Romberg Algorithm

The error of *Compound Trapezodial Rule*

$$
R_{T}[f,h] = \int_a^b f(x)\textbf{d}x - T_n(h) = \beta_2 h^2 + \beta_4 h^4 + ... + \beta_{2k} h^{2k} + E
$$

has the form of *Euler-Maclaruin's* series, and $\beta$ is not related to $h$. So it meets the conditions of *Richardson's Extrapolation*.

Let $T_0^{(k)}$ denote the *CTR* with $k$-time division, $T_m^{(l)}$ denote the $m$-th time of extrapolation, we can get:

$$
\begin{cases}
    T_0^{(0)} = \frac{b-a}{2} (f(a)+f(b)) \\
    T_0^{(k)} = \frac{1}{2} T_0^{(k-1)} + \frac{b-a}{2^k} \sum_{j=1}{2^{k-1}} f[a+\frac{2j-1}{2^k}(b-a)] \\
    T_m^{(l)} = \frac{4^m T_{m-1}^{(l+1)} - T_{m-1}^{(l)}}{4^m - 1}
\end{cases}
$$

Step by step, we can get the *Romberg Matrix*:

$$
\begin{bmatrix}
T_0^{(0)} \\
T_0^{(1)} & T_1^{(0)} \\
T_0^{(2)} & T_1^{(1)} & T_2^{(0)} \\
T_0^{(3)} & T_1^{(2)} & T_2^{(1)} & T_3^{(0)} \\
... & ... & ... & ... & ... 
\end{bmatrix}
$$

The convergence condition is given by $|T_m^{(0)} - T_{m-1}^{(0)}| < \epsilon$

In [5]:
import numpy as np

def romberg(f, a, b, epsilon):
    R = np.zeros((1, 1))
    h = b - a
    R[0, 0] = h * (f(a) + f(b)) / 2
    i = 1

    while True:
        i += 1
        h /= 2
        sum = 0
        for k in range(1, 2**(i-2) + 1):
            sum += f(a + (2*k - 1) * h)
        
        # expand the size of Romberg matrix
        R = np.pad(R, ((0, 1), (0, 1)), mode='constant', constant_values=0)
        R[i-1, 0] = 0.5 * R[i-2, 0] + h * sum

        for j in range(1, i):
            R[i-1, j] = R[i-1, j-1] + (R[i-1, j-1] - R[i-2, j-1]) / (4**j - 1)
        
        print(f"{i-1} iteration")
        print(f"Estimated value is: {R[i-1, i-1]}")

        if abs(R[i-1, i-1] - R[i-2, i-2]) < epsilon:
            break
    
    z = R[i-1, i-1]
    return R

def f(x):
    return np.sqrt(2) / ((1 + pow(np.sin(x), 2)) * np.sqrt(2 - pow(np.sin(x), 2)))

R = romberg(f, 0, np.pi, 1e-12)
print('Romberg matrix:')
print(R)

1 iteration
Estimated value is: 2.52815853058272
2 iteration
Estimated value is: 2.504634894746599
3 iteration
Estimated value is: 2.54830128782308
4 iteration
Estimated value is: 2.5463075029060844
5 iteration
Estimated value is: 2.546253223213955
6 iteration
Estimated value is: 2.5462547389715784
7 iteration
Estimated value is: 2.5462547334984587
8 iteration
Estimated value is: 2.5462547334993615
Romberg matrix:
[[3.14159265 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [2.68151706 2.52815853 0.         0.         0.         0.
  0.         0.         0.        ]
 [2.54995811 2.50610512 2.50463489 0.         0.         0.
  0.         0.         0.        ]
 [2.54625781 2.54502438 2.547619   2.54830129 0.         0.
  0.         0.         0.        ]
 [2.54625473 2.54625371 2.54633566 2.54631529 2.5463075  0.
  0.         0.         0.        ]
 [2.54625473 2.54625473 2.5462548  2.54625352 2.54625328 2.54625322
  0.         0.         0.        